# GeoCache: *Wine Spectator*'s Top 100 Wines, 1988-2020
List available online on *Wine Spectator*'s [Top 100 Lists web site](https://top100.winespectator.com/lists/).

## File Setup

In [23]:
# import and initialize main python libraries
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import bar_chart_race as bcr
import seaborn as sns

# import libraries for file navigation
import os
import shutil
import glob
from pandas_ods_reader import read_ods

# import other packages
from scipy import stats
from sklearn import linear_model

# import geo packages
import geopandas as gpd
import descartes
from shapely.geometry import Point, Polygon

# import Geopy packages
import geopy
from geopy.geocoders import Nominatim

In [24]:
# initialize vizualization set
sns.set(style="whitegrid", palette="colorblind", color_codes=True)
sns.mpl.rc("figure", figsize=(10, 6))

# Jupyter Notebook
%matplotlib inline

## Dataframe Exploration

In [25]:
# Note: save CSV files in UTF-8 format to preserve special characters.
df_Wine = pd.read_csv('./Wine_Hier00.csv')
df_GeoCache = pd.read_csv('./GeoCache.csv')
df_Flag = pd.read_csv('./emoji_list.csv')

In [26]:
df_Wine.shape

(3304, 25)

In [27]:
df_GeoCache.shape

(1228, 8)

In [28]:
df_Flag.shape

(19, 3)

In [29]:
df_Wine.sample(10)

,Review_Year,Rank,Vintage,Score,Price,Winemaker,Wine,Wine_Style,Grape_Blend,Blend_List,...,Best_Drink_from,Best_Drink_Through,Review,Hierarchy,Address,loc,point,lat,long,altitude
1507,2005.0,6,2003,96.0,56,Alban,Syrah Edna Valley Reva,Red,Shiraz | Syrah,NaN,...,2006.0,2012.0,"A rich, massive, densely flavored Syrah, packe...",Hierarchy_00,USA,United States,"(39.7837304, -100.4458825, 0.0)",39.783730,-100.445882,0.0
1963,2001.0,62,1999,91.0,15,Domaine de Beaurenard,Côtes du Rhône-Villages Rasteau,Red,Blend,Grenache - Syrah,...,2001.0,2010.0,Traditional-style red that's worth hunting dow...,Hierarchy_00,France,France,"(46.603354, 1.8883335, 0.0)",46.603354,1.888333,0.0
1954,2001.0,53,1999,92.0,27,Kendall-Jackson,Chardonnay Monterey Great Estates,White,Chardonnay,NaN,...,2001.0,2005.0,"Maintains a sense of elegance and finesse, eve...",Hierarchy_00,USA,United States,"(39.7837304, -100.4458825, 0.0)",39.783730,-100.445882,0.0
1411,2006.0,10,2004,95.0,50,Two Hands,Shiraz Barossa Valley Bella's Garden,Red,Shiraz | Syrah,NaN,...,2007.0,2020.0,"Dark, deep and impressively focused, a massive...",Hierarchy_00,Australia,Australia,"(-24.7761086, 134.755, 0.0)",-24.776109,134.755000,0.0
3268,1988.0,66,1985,93.0,25,Château La Croix,Pomerol,Red,Pomerol,NaN,...,1991.0,1993.0,Rich and powerful yet elegant and decadent wit...,Hierarchy_00,France,France,"(46.603354, 1.8883335, 0.0)",46.603354,1.888333,0.0
1698,2004.0,97,NV,90.0,18,Gloria Ferrer,Brut Sonoma County,Sparkling,Blend,"85% Pinot Noir, 15% Chardonnay",...,2004.0,2009.0,"An exceptional value. Round and full-bodied, w...",Hierarchy_00,USA,United States,"(39.7837304, -100.4458825, 0.0)",39.783730,-100.445882,0.0
2104,1999.0,3,1997,96.0,38,Isole & Olena,Toscana Cepparello,Red,Sangiovese,NaN,...,2002.0,NaN,This blockbuster of a Sangiovese shows layers ...,Hierarchy_00,Italy,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297,0.0
2358,1997.0,57,1995,92.0,28,Grgich Hills,Chardonnay Napa Valley,White,Chardonnay,NaN,...,NaN,NaN,"Brilliantly made. Ripe and elegant, with a spi...",Hierarchy_00,USA,United States,"(39.7837304, -100.4458825, 0.0)",39.783730,-100.445882,0.0
2674,1994.0,72,1990,91.0,26,Marcarini,Barolo Brunate,Red,Blend,Nebbiolo,...,2000.0,2004.0,"Ripe and exotic, layering some herbal notes of...",Hierarchy_00,Italy,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297,0.0
1935,2001.0,34,1999,93.0,26,Domaine de la Vieille Julienne,Châteauneuf-du-Pape,Red,Châteauneuf-du-Pape,NaN,...,2001.0,2010.0,"Supple, balanced and subtle. Traditional Châte...",Hierarchy_00,France,France,"(46.603354, 1.8883335, 0.0)",46.603354,1.888333,0.0


In [30]:
df_GeoCache.sample(10)

,Geography,Hierarchy,Address,loc,point,lat,long,altitude
1102,Anderson Valley,Hierarchy_03,"Mendocino County, North Coast, California, USA",NaN,NaN,NaN,NaN,NaN
523,Nahe,Hierarchy_01,"Nahe, Germany","Nahe, Itzstedt, Segeberg, Schleswig-Holstein, ...","(53.7942267, 10.13793, 0.0)",53.794227,10.137930,0.0
998,Blaye Côtes de Bordeaux,Hierarchy_03,"Blaye Côtes de Bordeaux, Côtes de Bordeaux, Bo...",NaN,NaN,NaN,NaN,NaN
411,Fronsac,Hierarchy_01,"Bordeaux, France","Bordeaux, Gironde, Nouvelle-Aquitaine, France ...","(44.841225, -0.5800364, 0.0)",44.841225,-0.580036,0.0
1218,Coombsville,Hierarchy_05,"Coombsville, Napa Valley, Napa County, North C...",NaN,NaN,NaN,NaN,NaN
554,Verdicchio dei Castelli di Jesi,Hierarchy_01,"Marche, Italy","Marche, Italia","(43.3458388, 13.1415872, 0.0)",43.345839,13.141587,0.0
437,Meursault,Hierarchy_01,"Burgundy, France","Bourgogne, France métropolitaine, France","(47.27808725, 4.222486304306048, 0.0)",47.278087,4.222486,0.0
350,Moulis-en-Medoc,Hierarchy_00,France,France,"(46.603354, 1.8883335, 0.0)",46.603354,1.888333,0.0
764,Chassagne-Montrachet Les Grandes Ruchottes,Hierarchy_02,"Côte de Beaune, Burgundy, France",NaN,NaN,NaN,NaN,NaN
933,Jumilla,Hierarchy_02,"Jumilla, Murcia, Spain","Jumilla, Altiplano, Región de Murcia, España","(38.4734708, -1.3285448, 0.0)",38.473471,-1.328545,0.0


In [31]:
df_Flag

,Address,Flag_and_Country,Flag
0,Argentina,🇦🇷 Argentina,🇦🇷
1,Australia,🇦🇺 Australia,🇦🇺
2,Austria,🇦🇹 Austria,🇦🇹
3,Canada,🇨🇦 Canada,🇨🇦
4,Chile,🇨🇱 Chile,🇨🇱
5,France,🇫🇷 France,🇫🇷
6,Germany,🇩🇪 Germany,🇩🇪
7,Greece,🇬🇷 Greece,🇬🇷
8,Hungary,🇭🇺 Hungary,🇭🇺
9,Israel,🇮🇱 Israel,🇮🇱


## Year over Year Analysis - Totals by Country

In [32]:
# pivot the Wine List by Country, Year, and count of Wine Spectator Scores
df_WineByCountry = pd.DataFrame({'count' : df_Wine.groupby(['Review_Year', 'Address'])['Score'].count()}).reset_index()

In [33]:
# Subset the GeoCache dataframe to exclude Geography column, return only unique values
df_GeoHierarchy = df_GeoCache.drop(columns = ['Geography'])
df_GeoHier00 = df_GeoHierarchy[ (df_GeoHierarchy['Hierarchy'] == 'Hierarchy_00') ]
df_GeoHier00 = df_GeoHier00.drop_duplicates()

In [34]:
# Join GeoCache dataframe to Wine List dataframe to obtain 
df_WineByCountry = pd.merge(df_WineByCountry, df_GeoHier00, on = 'Address', how = 'left')
df_WineByCountry = pd.merge(df_WineByCountry, df_Flag, on = 'Address', how = 'left')
df_WineByCountry = df_WineByCountry.sort_values(['Review_Year', 'count'], ascending = [True, False])

In [35]:
# represent Review Year as a Whole Number
df_WineByCountry['Review_Year'] = df_WineByCountry['Review_Year'].astype(int)

## Bar Chart Race: Count of Scores by Country by Year

In [36]:
# pivot the data to facilitate bar chart race
df_BRC_Wine = df_WineByCountry.pivot(index = 'Review_Year', columns = 'Address', values = 'count')

# replace 'NaN' results in chart with 0 to facilitate graphing
df_BRC_Wine.fillna(value = 0, inplace = True)

df_BRC_Wine

Address,Argentina,Australia,Austria,Canada,Chile,France,Germany,Greece,Hungary,Israel,Italy,Lebanon,New Zealand,Portugal,South Africa,Spain,USA,Ukraine,Uruguay
Review_Year,,,,,,,,,,,,,,,,,,,
1988,0.0,1.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,15.0,1.0,1.0,0.0,0.0,3.0,34.0,0.0,0.0
1989,0.0,4.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,1.0,61.0,0.0,0.0
1990,0.0,2.0,0.0,0.0,0.0,24.0,3.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,5.0,51.0,0.0,0.0
1991,0.0,2.0,0.0,0.0,0.0,39.0,6.0,0.0,0.0,0.0,13.0,0.0,1.0,1.0,0.0,1.0,37.0,0.0,0.0
1992,0.0,4.0,0.0,0.0,1.0,42.0,0.0,0.0,0.0,0.0,11.0,0.0,1.0,0.0,0.0,5.0,36.0,0.0,0.0
1993,0.0,1.0,0.0,0.0,0.0,44.0,3.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0
1994,0.0,5.0,0.0,0.0,0.0,16.0,1.0,0.0,0.0,0.0,14.0,0.0,2.0,5.0,0.0,3.0,54.0,0.0,0.0
1995,1.0,8.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,12.0,0.0,1.0,3.0,2.0,3.0,56.0,0.0,0.0
1996,0.0,16.0,0.0,0.0,2.0,16.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,1.0,1.0,1.0,56.0,0.0,0.0


In [37]:
# bar chart race - visualize in Jupyter Notebook
bcr.bar_chart_race(
    df = df_BRC_Wine,
    filename = None,
    orientation = 'h',
    sort = 'desc',
    #n_bars = 4,
    fixed_order = True,
    fixed_max = True,
    steps_per_period = 10,
    interpolate_period = False,
    label_bars = True,
    bar_size = 0.95,
    #period_label = {'x': 0.99, 'y': 0.25, 'ha': 'right', 'va': 'center'},
    period_fmt = 'Review Year - {x:.0f}',
    period_length = 1000,
    figsize = (6, 4),
    dpi = 144,
    cmap = 'dark12',
    title = 'Representation in Wine Spectator Top 100 Wines\nCount of Scores by Country by Year',
    title_size = '',
    #bar_label_size = 7,
    #tick_label_size = 7,
    scale = 'linear',
    writer = None,
    fig = None,
    #bar_kwargs: {'alpha': 0.7},
    filter_column_colors = True
)

In [38]:
# bar chart race - visualize in Jupyter Notebook
bcr.bar_chart_race(
    df = df_BRC_Wine,
    filename = 'BRC_Count_Country_Year.mp4',
    orientation = 'h',
    sort = 'desc',
    #n_bars = 4,
    fixed_order = True,
    fixed_max = True,
    steps_per_period = 10,
    interpolate_period = False,
    label_bars = True,
    bar_size = 0.95,
    #period_label = {'x': 0.99, 'y': 0.25, 'ha': 'right', 'va': 'center'},
    period_fmt = 'Review Year - {x:.0f}',
    period_length = 1000,
    figsize = (6, 4),
    dpi = 144,
    cmap = 'dark12',
    title = 'Representation in Wine Spectator Top 100 Wines\nCount of Scores by Country by Year',
    title_size = '',
    #bar_label_size = 7,
    #tick_label_size = 7,
    scale = 'linear',
    writer = None,
    fig = None,
    #bar_kwargs: {'alpha': 0.7},
    filter_column_colors = True
)